**程序运行需要建立ampqstorm 服务器，电脑上没有改程序，无法建立连接，没有运行示例程序。**

### Section 117.1: How to consume messages from RabbitMQ

In [2]:
from amqpstorm import Connection

In [3]:
def on_message(message):
    """This function is called on message received.
    :param message: Delivered message.
    :return:
    """
    print("Message:", message.body)
    # Acknowledge that we handled the message without any issues.
    message.ack()
    # Reject the message.
    # message.reject()
    # Reject the message, and put it back in the queue.
    # message.reject(requeue=True)

In [ ]:
connection = Connection('127.0.0.1', 'guest', 'guest')

In [ ]:
channel = connection.channel()

In [ ]:
channel.basic.consume(callback=on_message, queue='simple_queue', no_ack=False)

In [ ]:
channel.start_consuming(to_tuple=False)

### Section 117.2: How to publish messages to RabbitMQ

In [ ]:
from amqpstorm import Connection
from amqpstorm import Message

In [ ]:
connection = Connection('127.0.0.1', 'guest', 'guest')

In [ ]:
channel = connection.channel()

In [ ]:
# Message Properties.
properties = {
'content_type': 'text/plain',
'headers': {'key': 'value'}
}

In [ ]:
# Create the message.
message = Message.create(channel=channel, body='Hello World!', properties=properties)

In [ ]:
message.publish(routing_key='simple_queue')

### Section 117.3: How to create a delayed queue in RabbitMQ

In [ ]:
channel.queue.bind(exchange='amq.direct', routing_key='hello', queue='hello')

In [ ]:
Next we need to configure our delay channel to forward messages to the main queue once they have expired.

In [ ]:
delay_channel.queue.declare(queue='hello_delay', durable=True, arguments={
'x-message-ttl': 5000,
'x-dead-letter-exchange': 'amq.direct',
'x-dead-letter-routing-key': 'hello'
})

1. x-message-ttl (Message - Time To Live)

This is normally used to automatically remove old messages in the queue after a specific duration, but by
adding two optional arguments we can change this behaviour, and instead have this parameter determine in
milliseconds how long messages will stay in the delay queue.
2. x-dead-letter-routing-key

This variable allows us to transfer the message to a different queue once they have expired, instead of the
default behaviour of removing it completely.
3. x-dead-letter-exchange

This variable determines which Exchange used to transfer the message from hello_delay to hello queue.

**Publishing to the delay queue**

In [ ]:
delay_channel.basic.publish(exchange='',
routing_key='hello_delay',
body='test',
properties={'delivery_mod': 2})

In [ ]:
from amqpstorm import Connection
connection = Connection('127.0.0.1', 'guest', 'guest')
# Create normal 'Hello World' type channel.
channel = connection.channel()
channel.confirm_deliveries()
channel.queue.declare(queue='hello', durable=True)
# We need to bind this channel to an exchange, that will be used to transfer
# messages from our delay queue.
channel.queue.bind(exchange='amq.direct', routing_key='hello', queue='hello')
# Create our delay channel.
delay_channel = connection.channel()
delay_channel.confirm_deliveries()
# This is where we declare the delay, and routing for our delay channel.
delay_channel.queue.declare(queue='hello_delay', durable=True, arguments={
    'x-message-ttl': 5000, # Delay until the message is transferred in milliseconds.
    'x-dead-letter-exchange': 'amq.direct', # Exchange used to transfer the message from A to B.
    'x-dead-letter-routing-key': 'hello' # Name of the queue we want the message transferred to.
})
delay_channel.basic.publish(exchange='',
    routing_key='hello_delay',
    body='test',
    properties={'delivery_mode': 2})
print("[x] Sent")

In [7]:
help(Connection)

Help on class Connection in module amqpstorm.connection:

class Connection(amqpstorm.base.Stateful)
 |  RabbitMQ Connection.
 |  
 |  Method resolution order:
 |      Connection
 |      amqpstorm.base.Stateful
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, exception_type, exception_value, _)
 |  
 |  __init__(self, hostname, username, password, port=5672, **kwargs)
 |      :param str hostname: Hostname
 |      :param str username: Username
 |      :param str password: Password
 |      :param int port: Server port
 |      :param str virtual_host: Virtual host
 |      :param int heartbeat: RabbitMQ Heartbeat interval
 |      :param int|float timeout: Socket timeout
 |      :param bool ssl: Enable SSL
 |      :param dict ssl_options: SSL kwargs (from ssl.wrap_socket)
 |      :param bool lazy: Lazy initialize the connection
 |      
 |      :raises AMQPConnectionError: Raises if the connection
 |                                   enc